In [10]:
import pandas as pd
import numpy as np
import math
import heapq
import time
import folium

In [11]:
cities_df = pd.read_csv("Data/Cities_of_SriLanka.csv")


# Convert to dictionary for easy lookup
city_coords = {row['name_en']: (row['latitude'], row['longitude']) for _, row in cities_df.iterrows()}

In [12]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371 # Earth radius in km
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    return 2 * R * math.atan2(math.sqrt(a), math.sqrt(1-a))

In [13]:
K = 5  # number of neighbors
cities = list(city_coords.keys())
graph = {city: [] for city in cities}

for city in cities:
    lat1, lon1 = city_coords[city]
    distances = []
    for other in cities:
        if city == other:
            continue
        lat2, lon2 = city_coords[other]
        d = haversine(lat1, lon1, lat2, lon2)
        distances.append((d, other))
    distances.sort()
    for d, neighbor in distances[:K]:
        graph[city].append((neighbor, d))

In [14]:
def dijkstra(graph, start, end):
    pq = [(0, start, [])]
    visited = set()
    while pq:
        (dist, node, path) = heapq.heappop(pq)
        if node in visited:
            continue
        visited.add(node)
        path = path + [node]
        if node == end:
            return dist, path
        for neighbor, weight in graph[node]:
            if neighbor not in visited:
                heapq.heappush(pq, (dist+weight, neighbor, path))
    return float("inf"), []

In [15]:
source = "Yakkala"
destination = "Hungama"

print(f"Finding shortest path from {source} to {destination}\n")

# Dijkstra
start_time = time.time()
dist, path = dijkstra(graph, source, destination)
print("Dijkstra:", path, "Distance:", round(dist,2), "km", "Time:", round(time.time()-start_time,4), "s")

Finding shortest path from Yakkala to Hungama

Dijkstra: [] Distance: inf km Time: 0.0003 s


In [18]:
if path:
    lat, lon = city_coords[source]
    m = folium.Map(location=[lat, lon], zoom_start=8)

    # Add cities to map
    for city in path:
        lat, lon = city_coords[city]
        folium.Marker([lat, lon], popup=city, icon=folium.Icon(color="blue")).add_to(m)

    # Draw path line
    coords = [(city_coords[city][0], city_coords[city][1]) for city in path]
    folium.PolyLine(coords, color="red", weight=3, opacity=0.8).add_to(m)

    # Save map
    m.save("shortest_path_map.html")
    print("Map saved as shortest_path_map.html")
